In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SVM:
    def __init__(self, C=1.0):
        # C = Termo de penalidade (Controla o trade-off entre margem e erro de treinamento)
        # Em otimização restrita, atua de forma similar a um Multiplicador de Lagrange
        self.C = C
        self.w = None # Vetor de pesos (weights)
        self.b = None # Viés (bias)
        
    def calcular_funcao_custo(self, w, b, X, y):
        """
        Calcula a 'Hinge Loss' + Regularização.
        Isso representa a Função Objetivo que queremos minimizar.
        """
        # Termo de Regularização (Maximizar a margem geometricamente)
        # 0.5 * ||w||^2
        regularizacao = 0.5 * np.sum(w ** 2)

        # Cálculo vetorial dos "Resíduos"
        # y_i * (w . x_i + b)
        distancia_funcional = y * (np.dot(X, w.T).flatten() + b)
        
        # Hinge Loss: max(0, 1 - y_i(w.x_i + b))
        # Penaliza apenas se o ponto estiver dentro da margem ou classificado errado
        perda_hinge = np.maximum(0, 1 - distancia_funcional)
        
        # Custo Total = Regularização + C * Soma dos Erros
        custo_total = regularizacao + self.C * np.sum(perda_hinge)
        return custo_total

    def fit(self, X, y, tamanho_lote=100, taxa_aprendizado=0.001, epocas=1000):
        """
        Treina o modelo usando Descida do Gradiente (Gradient Descent - Steepest Descent).
        """
        n_amostras, n_features = X.shape

        # Inicialização dos parâmetros (w e b)
        self.w = np.zeros((1, n_features))
        self.b = 0
        
        historico_custo = []

        # Loop de Otimização (Iterações/Épocas)
        for i in range(epocas):
            
            # Embaralhamento dos dados (Shuffling)
            # Essencial para a convergência do Gradiente
            indices = np.arange(n_amostras)
            np.random.shuffle(indices)
            X_embaralhado = X[indices]
            y_embaralhado = y[indices]

            # Monitoramento da Função Objetivo
            custo_atual = self.calcular_funcao_custo(self.w, self.b, X, y)
            historico_custo.append(custo_atual)

            # Iteração sobre Batch
            for inicio in range(0, n_amostras, tamanho_lote):
                fim = inicio + tamanho_lote
                X_batch = X_embaralhado[inicio:fim]
                y_batch = y_embaralhado[inicio:fim]

                # --- Cálculo do Gradiente (Derivadas Parciais) ---
                
                # Verificamos a condição da margem: y * (w.x + b) < 1
                condicao = y_batch * (np.dot(X_batch, self.w.T).flatten() + self.b)
                
                # Identifica índices onde o modelo errou ou violou a margem
                indices_violacao = np.where(condicao < 1)[0]

                # 1. Gradiente da Regularização (sempre aplicado)
                # Derivada de 0.5*w^2 é w
                grad_w = self.w 
                grad_b = 0

                # 2. Gradiente da Perda Hinge (aplicado apenas onde há violação)
                if len(indices_violacao) > 0:
                    X_violacao = X_batch[indices_violacao]
                    y_violacao = y_batch[indices_violacao].reshape(-1, 1)

                    # Derivada parcial em relação a w: -C * y * x
                    grad_w_erro = -self.C * np.dot(y_violacao.T, X_violacao)
                    
                    # Derivada parcial em relação a b: -C * y
                    grad_b_erro = -self.C * np.sum(y_violacao)
                    
                    # Soma os componentes do gradiente (Regularização + Erro)
                    # Nota: Como a atualização abaixo é w = w - lr * grad,
                    # e o termo do erro é negativo, somamos ele aqui para que a subtração
                    # na atualização resulte em uma soma (descida correta).
                    # A lógica expandida seria: w_novo = w_velho - taxa * (w_velho - C*y*x)
                    
                    # Atualização dos Pesos (Descida do Gradiente)
                    self.w = self.w - taxa_aprendizado * (self.w + grad_w_erro)
                    self.b = self.b - taxa_aprendizado * grad_b_erro
                
                else:
                    # Se não houver violação, apenas o termo de regularização atua (decaimento de peso)
                    self.w = self.w - taxa_aprendizado * self.w

        return self.w, self.b, historico_custo

    def predict(self, X):
        # Predição baseada no sinal da projeção
        predicao = np.dot(X, self.w.T) + self.b
        return np.sign(predicao).flatten()

# --- Visualização ---

def visualizar_svm(modelo, X, y):
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='winter', s=50, alpha=0.6)

    ax = plt.gca()
    xlim = ax.get_xlim()
    
    xx = np.linspace(xlim[0], xlim[1], 30)
    
    # Parâmetros otimizados
    w = modelo.w.flatten()
    b = modelo.b
    
    # Hiperplano de Decisão (Decision Boundary): w.x + b = 0
    yy = (-w[0] * xx - b) / w[1]
    
    # Margens (+1 e -1): w.x + b = 1  e  w.x + b = -1
    margem_pos = (1 - w[0] * xx - b) / w[1]
    margem_neg = (-1 - w[0] * xx - b) / w[1]

    plt.plot(xx, yy, 'k-', label='Hiperplano de Separação')
    plt.plot(xx, margem_pos, 'k--', linewidth=0.5, label='Margem (+1)')
    plt.plot(xx, margem_neg, 'k--', linewidth=0.5, label='Margem (-1)')
    
    plt.legend()
    plt.title(f"SVM Linear (C={modelo.C})")
    plt.show()

if __name__ == "__main__":
    # Geração de dados sintéticos
    X = np.random.randn(200, 2) * 2
    # Rótulos baseados em uma separação linear simples
    y = np.array([1 if x[0] + x[1] > 0 else -1 for x in X])
    
    # Instância e Treinamento
    svm = SVM(C=1.0)
    w_final, b_final, historico = svm.fit(X, y, tamanho_lote=20, epocas=500)
    
    visualizar_svm(svm, X, y)
    
    plt.plot(historico)
    plt.title("Convergência da Função Objetivo (Descida do Gradiente)")
    plt.xlabel("Épocas")
    plt.ylabel("Custo")
    plt.show()